# Yet another `planning` tutorials

This is Tutorial about `Robot path planning` methods. I will implement Dijkstra, A-star, RRT, Particle filters, and so on.

## Dijkstra

In [1]:
import heapq

def dijkstra(graph, start):
    distances = {node: float('inf') for node in graph}
    distances[start] = 0    # the start node is zero.

    queue = []
    heapq.heappush(queue, [distances[start], start])    # begin searching at start node.

    while queue:
        # pop at the most right ones.
        curr_dist, curr_dest = heapq.heappop(queue)    # get node, distance

        if distances[curr_dest] < curr_dist:    # if the destination dist is longer, no longer to search.
            continue

        for new_dest, new_dist in graph[curr_dest].items():
            distance = curr_dist + new_dist     # the overall distance when pass this node.

            if distance < distances[new_dest]:  # update when smaller than the known node distance.
                distances[new_dest] = distance  
                heapq.heappush(queue, [distance, new_dest]) # for calculate next nearset, push at queue
    
    return distances


In [2]:
graph = {
    'A': {'B': 8, 'C': 1, 'D': 2},
    'B': {},
    'C': {'B': 5, 'D': 2},
    'D': {'E': 3, 'F': 5},
    'E': {'F': 1},
    'F': {'A': 5}
}

graph   # dict type

{'A': {'B': 8, 'C': 1, 'D': 2},
 'B': {},
 'C': {'B': 5, 'D': 2},
 'D': {'E': 3, 'F': 5},
 'E': {'F': 1},
 'F': {'A': 5}}

In [3]:
dijkstra(graph, 'A')

{'A': 0, 'B': 6, 'C': 1, 'D': 2, 'E': 5, 'F': 6}

## A Star

### `A star` pseudo code
// A* (star) Pathfinding

// Initialize both open and closed list
let the openList equal empty list of nodes
let the closedList equal empty list of nodes

// Add the start node
put the startNode on the openList (leave it's f at zero)

// Loop until you find the end
while the openList is not empty
    
    // Get the current node
    let the currentNode equal the node with the least f value
    remove the currentNode from the openList
    add the currentNode to the closedList
    
    // Found the goal
    if currentNode is the goal
        Congratz! You've found the end! Backtrack to get path

    // Generate children
    let the children of the currentNode equal the adjacent nodes
    
    for each child in the children
        // Child is on the closedList
        if child is in the closedList
            continue to beginning of for loop
        // Create the f, g, and h values
        child.g = currentNode.g + distance between child and current
        child.h = distance from child to end
        child.f = child.g + child.h
        // Child is already in openList
        if child.position is in the openList's nodes positions
            if the child.g is higher than the openList node's g
                continue to beginning of for loop
        // Add the child to the openList
        add the child to the openList

In [4]:
import numpy as np

class Node:
    def __init__(self, parent=None, position=None):
        self.parent = parent        # 이전 노드
        self.position = position    # 현재 위치

        self.f  = 0
        self.g  = 0
        self.h  = 0
    
    def __eq__(self, other):
        return self.position == other.position


In [5]:
def calculate_heuristic(curr_node, dest_node):
    distance = np.sqrt((dest_node.position[0] - curr_node.position[0]) ** 2 + (dest_node.position[1] - curr_node.position[1]) ** 2)

    return distance

In [6]:
start_node = Node(None, (0, 0))

start_node.position

end_node = Node(None, (3, 4))

end_node.position

calculate_heuristic(start_node, end_node)

5.0

In [8]:
def astar(maze, start, end):
    start_node  = Node(None, start) # start position: (x, y)
    end_node    = Node(None, end)   # end position: (x, y)

    openlist    = []
    closedlist  = []

    openlist.append(start_node)

    while openlist:
        # Initialize current node.
        current_node    = openlist[0]
        current_idx     = 0

        for idx, item in enumerate(openlist):
            # search openlist nodes that have minimum cost F.
            if item.f < current_node.f:
                current_node    = item
                current_idx     = idx

        # delete in openlist, and append in closedlist.
        openlist.pop(current_idx)
        closedlist.append(current_node)

        # when the search is finished.
        if current_node == end_node:
            path = []
            current = current_node

            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1]   # return reversed.

        
        # generate children
        children = []

        # Adjacent squares
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]: 
            # get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # check the position is within the maze.
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue

            # collision check.
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            new_node = Node(current_node, node_position)

            # Append
            children.append(new_node)

        for child in children:
            for closed_child in closedlist:
                if child == closed_child:
                    continue
            
            child.g = current_node.g + 1
            # child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            child.h = calculate_heuristic(child, end_node)
            child.f = child.g + child.h

            for open_node in openlist:
                if child == open_node and child.g > open_node.g:
                    continue

            openlist.append(child)        

In [9]:
# define Maze.
maze = [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

start = (0, 0)
end = (7, 6)

path = astar(maze, start, end)
print(path)


[(0, 0), (1, 1), (2, 2), (3, 3), (4, 3), (5, 4), (6, 5), (7, 6)]


## RRT

In [ ]:
import random

class RRT:
    """
        Class for RRT Planning.
    
    """

    class AreaBounds:
        def __init__(self, area):
            self.xmin   = float(area[0])
            self.xmax   = float(area[1])
            self.ymin   = float(area[2])
            self.ymax   = float(area[3])


In [ ]:
class Node:
    """
        RRT Node.
    """

    def __init__(self, x, y):
        self.x  = x
        self.y  = y
        self.path_x = []
        self.path_y = []
        self.parent = None



In [ ]:
goal_sample_rate = 5

def get_random_node():
    if random.randint(0, 100) > goal_sample_rate:
        random_node = 

## RRT-Star